# NIKA2 Run preparation

This notebook shows how to compute the primary calibrator fluxes for a NIKA2 run. These are the planets
* Mars
* Uranus
* Neptune

For each calibrator, we will write a text file containing the values

In [1]:
import photometry as pt
from astropy.time import Time
from astropy import units as u
import numpy as np
import os

In [2]:
# get the planet objects
uranus = pt.GiantPlanet('Uranus', model_version='esa4')
neptune = pt.GiantPlanet('Neptune', model_version='esa5')
mars = pt.Mars()
# and the NIKA2 passbands
nika2mm = pt.Passband(file='2mm.NIKA2.pb')
nika1mmH = pt.Passband(file='1mmH.NIKA2.pb')
nika1mmV = pt.Passband(file='1mmV.NIKA2.pb')

In [3]:
# define the variables defining the run
output_dir = '/Users/haussel/work/nika2/commissioning/run27/'
run = 27
start_date = '2018-01-15'
end_date = '2018-01-23'
start_day = 15
end_day = 23

In [4]:
# Get one value per day
dates = Time(["2018-01-{:02d}".format(x) for x in np.arange(start_day, end_day+1)], 
             format='iso')

In [5]:
# compute the Neptune and Uranus Fluxes at the reference frequencies of each band
neptune.set_dates(dates)
uranus.set_dates(dates)
spec_uranus = uranus.spectral_irradiance()
spec_neptune = neptune.spectral_irradiance()

fluxes_1mm_uranus= spec_uranus.fnu_nu(nika1mmH.xref(u.GHz)).to(u.Jy)
fluxes_1mm_neptune = spec_neptune.fnu_nu(nika1mmH.xref(u.GHz)).to(u.Jy)
fluxes_2mm_uranus = spec_uranus.fnu_nu(nika2mm.xref(u.GHz)).to(u.Jy)
fluxes_2mm_neptune = spec_neptune.fnu_nu(nika2mm.xref(u.GHz)).to(u.Jy)

In [6]:
# compute Mars fluxes at the reference frequencies for each run
mars.set_beam('Iram30m')
mars.set_instrument('NIKA2')

<Quantity [ 150.,  260., 1600., 3200.] GHz>

In [7]:
Fnu150GHzMars = np.zeros(len(dates))
Fnu260GHzMars = np.zeros(len(dates))
for i, date in enumerate(dates):
    Fnu150GHzMars[i] = mars.fnu(150 * u.GHz, date).value
    Fnu260GHzMars[i] = mars.fnu(260 * u.GHz, date).value

In [8]:
# for output, get days in a NIKA2 scan name  format
days = ["201801{:02d}".format(x) for x in np.arange(start_day, end_day+1)]

In [9]:
# write the output for uranus
out_uranus = os.path.join(output_dir, 'uranus_run{}.txt'.format(run))
with open(out_uranus, 'w') as fout:
    fout.write("# Date Fnu_150GHz   Fnu_260GHz")
    for i, day in enumerate(days):
        fout.write("{} {:5.3f} {:5.3f}\n".format(day, fluxes_2mm_uranus[i].value, 
                                                 fluxes_1mm_uranus[i].value))
fout.close()

In [10]:
# write the output for neptune
out_neptune = os.path.join(output_dir, 'neptune_run{}.txt'.format(run))
with open(out_neptune, 'w') as fout:
    fout.write("# Date Fnu_150GHz   Fnu_260GHz")
    for i, day in enumerate(days):
        fout.write("{} {:5.3f} {:5.3f}\n".format(day, fluxes_2mm_neptune[i].value, 
                                                 fluxes_1mm_neptune[i].value))
fout.close()

In [11]:
# write the output for mars
out_mars = os.path.join(output_dir, 'mars_run{}.txt'.format(run))
with open(out_mars, 'w') as fout:
    fout.write("# Date Fnu_150GHz   Fnu_260GHz")
    for i, day in enumerate(days):
        fout.write("{} {:5.3f} {:5.3f}\n".format(day, Fnu150GHzMars[i], 
                                                 Fnu260GHzMars[i]))
fout.close()

In [12]:
# All done. Email the files to Nicolas Ponthieu